## 1A INDEXING


In [2]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

from langchain.schema.runnable import RunnableSequence, RunnableParallel ,RunnableLambda , RunnableBranch , RunnablePassthrough

model = ChatOllama(
    model="qwen2.5:1.5b",  # or "llama2", "mistral", etc.
    base_url="http://127.0.0.1:11434"
)
model1=ChatOpenAI()

from langchain_core.messages import SystemMessage, HumanMessage , AIMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


load_dotenv()
model=ChatOpenAI()


In [3]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [4]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled

video_id = "LiG1DcqYjXU"

transcript_list = YouTubeTranscriptApi()
obj=transcript_list.fetch(video_id)






In [5]:
obj

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='Krishnavatara was considered as a complete\navatara or a complete incarnation of god.', start=0.96, duration=7.8), FetchedTranscriptSnippet(text='The consciousness that we refer to as Krishna,\nif we have to be touched by this we need Leela.', start=8.76, duration=9.82), FetchedTranscriptSnippet(text='Leela does not mean just dancing with somebody.', start=18.59, duration=3.01), FetchedTranscriptSnippet(text='You’re willing to dance with death.', start=21.6, duration=1.91), FetchedTranscriptSnippet(text='You’re willing to dance with life.', start=23.51, duration=2.37), FetchedTranscriptSnippet(text='SUPER: Krishna: A Complete Incarnation of the Divine', start=25.88, duration=8.14), FetchedTranscriptSnippet(text='Sadhguru: Very, very few human beings ever get to be\nplaced in as complex situations as Krishna was placed.', start=34.02, duration=14.24), FetchedTranscriptSnippet(text='In one human being’s life going through\nthe va

In [6]:
# Extract the text cleanly
text_chunks = [snippet.text for snippet in obj.snippets]
joined_text = " ".join(text_chunks)


In [7]:
prompt1=PromptTemplate(
    template="""
You are a text-cleaning bot. Given a raw transcript snippet, return only the corrected transcript without adding any explanation, heading, or commentary. Do not say anything extra or dont include backslashed or forward slashes.

Transcript:
{topic}

Cleaned Transcript:
""",
    input_variables=["topic"]
)


In [8]:
parser=StrOutputParser()

In [9]:
chain=RunnableSequence(prompt1,model,parser)
result=chain.invoke({'topic':obj})

In [10]:
result

'Krishnavatara was considered as a complete avatara or a complete incarnation of god. The consciousness that we refer to as Krishna, if we have to be touched by this we need Leela. Leela does not mean just dancing with somebody. You’re willing to dance with death. You’re willing to dance with life. Krishna: A Complete Incarnation of the Divine. Sadhguru: Very, very few human beings ever get to be placed in as complex situations as Krishna was placed. In one human being’s life going through the variety of situations that he went through is, generally it doesn’t happen to most, most human beings. And in all these situations that he was faced with the clarity of vision that he exhibited and the certainty about his ability to discern as to what is true and what is not true. The clarity that he brought to people around him, people began to experience him godlike. So he was proclaimed as an incarnation and a complete incarnation. For those of you who are familiar with this, they talk about n

## 1B INDEXING 

In [11]:
splitter = RecursiveCharacterTextSplitter(chunk_size=150,chunk_overlap=20)
chunks=splitter.create_documents([result])

In [12]:
len(chunks)

56

In [13]:
chunks[0]

Document(page_content='Krishnavatara was considered as a complete avatara or a complete incarnation of god. The consciousness that we refer to as Krishna, if we have to be')

## Embedding generation and storing in vectore store

In [14]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectore_store=FAISS.from_documents(chunks , embeddings)

In [15]:
vectore_store.index_to_docstore_id

{0: '4c7f0ac7-9fe7-4045-b2c6-40e00dcceee3',
 1: 'db10e7fb-1935-42b5-9955-a136e4296eae',
 2: '2734c712-a1f0-49be-8284-e96af344ebfa',
 3: '6a57e485-9a38-4091-8dc5-d01d77bfe5fd',
 4: '27893e87-8cdc-4049-ad63-4619b5f0e579',
 5: 'f2806da3-1e39-4c4f-9191-1b11c8aa8b42',
 6: 'c3493169-7ec4-43cc-aecb-37f435c5775c',
 7: '3ff5c20e-27a5-4fdb-bd4f-176a512d5a86',
 8: 'ba5738b6-c383-4c98-a5b3-97139176016b',
 9: 'e52410d7-a8e3-4ce1-9e0c-48db472b37f7',
 10: 'fdd14fc5-f81b-48e6-a375-bd0c19685d51',
 11: '4893d528-aecb-458a-8371-15c25cbef936',
 12: '54a250aa-bea5-4a51-b11f-9202d4d629f7',
 13: 'f8d981f0-91f4-4e28-b1f8-d3164797f79d',
 14: '10e0b59b-a6a9-4fce-a12f-f8844a9583e7',
 15: '1041b71a-9952-40c0-9d9d-7050da5ba427',
 16: '05c16fe1-ac36-4132-aa15-7384a543c37d',
 17: '50deeedf-89d6-4317-8a0f-c6edf024857a',
 18: '574570cf-4e1d-478f-aa6a-d0782fed960c',
 19: '5de7d712-d6fe-4217-9aea-e932683c6ada',
 20: 'ccac7034-a8fd-4bd8-9d96-4aeb5486e2f7',
 21: 'db577bee-6359-4762-a585-d7291564bd19',
 22: '7b0b2389-2383-

## 2 Retrievar

In [16]:
retriever = vectore_store.as_retriever(search_type="similarity",search={'k':2})

In [17]:
retriever.invoke('who is krishna')

[Document(page_content='we say Krishna, the essence of who he is, he is an irrepressible child, a terrible prankster, an enchanting flute player, a graceful dancer, an'),
 Document(page_content='people attached to him the whole culture evolved around this identity of Krishna being a complete incarnation of the divine. When we say Krishna, the'),
 Document(page_content='the most colorful incarnation. Krishna is seen and perceived and understood and experienced in many, many different ways by different people.'),
 Document(page_content='Krishnavatara was considered as a complete avatara or a complete incarnation of god. The consciousness that we refer to as Krishna, if we have to be')]

## 3. augmentation

In [18]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [ ]:
question          = " who is krishna"
retrieved_docs    = retriever.invoke(question)

In [20]:
retrieved_docs

[Document(page_content='we say Krishna, the essence of who he is, he is an irrepressible child, a terrible prankster, an enchanting flute player, a graceful dancer, an'),
 Document(page_content='people attached to him the whole culture evolved around this identity of Krishna being a complete incarnation of the divine. When we say Krishna, the'),
 Document(page_content='the most colorful incarnation. Krishna is seen and perceived and understood and experienced in many, many different ways by different people.'),
 Document(page_content='Akrura, an uncle of Krishna, a wise and saintly being expressed his perception of Krishna this way, “Krishna, when I look at him, this strange young')]

In [21]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

'we say Krishna, the essence of who he is, he is an irrepressible child, a terrible prankster, an enchanting flute player, a graceful dancer, an\n\npeople attached to him the whole culture evolved around this identity of Krishna being a complete incarnation of the divine. When we say Krishna, the\n\nthe most colorful incarnation. Krishna is seen and perceived and understood and experienced in many, many different ways by different people.\n\nAkrura, an uncle of Krishna, a wise and saintly being expressed his perception of Krishna this way, “Krishna, when I look at him, this strange young'

In [22]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [23]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      we say Krishna, the essence of who he is, he is an irrepressible child, a terrible prankster, an enchanting flute player, a graceful dancer, an\n\npeople attached to him the whole culture evolved around this identity of Krishna being a complete incarnation of the divine. When we say Krishna, the\n\nthe most colorful incarnation. Krishna is seen and perceived and understood and experienced in many, many different ways by different people.\n\nAkrura, an uncle of Krishna, a wise and saintly being expressed his perception of Krishna this way, “Krishna, when I look at him, this strange young\n      Question:  who is krishna\n    ")

##  Generations

In [24]:
answer = model.invoke(final_prompt)
print(answer.content)

Krishna is an irrepressible child, a terrible prankster, an enchanting flute player, a graceful dancer, and the most colorful incarnation. Krishna is seen and perceived and understood and experienced in many different ways by different people.
